# Skalierbare Methoden der Künstlichen Intelligenz
Charlotte Debus (charlotte.debus@kit.edu)\
Markus Götz (markus.goetz@kit.edu)\
Marie Weiel (marie.weiel@kit.edu)
## Übung 2 am 22.11.22: Parallele k-Means-Clusteranalyse
In der zweiten Übung beschäftigen wir uns mit der k-Means-Clusteranalyse und möglichen Parallelisierungsansätzen (siehe Vorlesung vom 10.11.22). Dazu verwenden wir den [Cityscapes](https://www.cityscapes-dataset.com/)-Datensatz. Dieser Datensatz enthält Stereovideosequenzen von Straßenszenen aus 50 verschiedenen Städten mit feinkörnigen Annotationen auf Pixelebene von 5000 hochaufgelösten Bildern.
Jedes dieser Bilder besteht aus 2048 x 1024 Pixeln mit drei 256-Bit RGB-Farbkanälen pro Pixel, die in einer "Short-Fat"-Matrix mit 5000 x 6 291 456 seriellen Einträgen zusammengefasst sind:  
5000 Bilder x (3 Kanäle x 2048 Pixel x 1024 Pixel) = 5000 x 6 291 456  
Für unsere Aufgabe benutzen wir 300 dieser Samples. Sie finden diese auf dem bwUniCluster im Workspace `VL_ScalableAI` unter folgendem Pfad:  
`/pfs/work7/workspace/scratch/ku4408-VL_ScalableAI/data/cityscapes_300.h5`
### Aufgabe 1
Untenstehend finden Sie eine serielle Implementierung des k-Means-Algorithmus in Python 3 unter Verwendung der Programmbibliothek für maschinelles Lernen [PyTorch](https://pytorch.org/). 
Führen Sie den Code auf einer CPU bzw. GPU auf dem bwUniCluster aus. Beachten Sie dabei, dass der Code für die GPU-Nutzung angepasst werden muss. Vergleichen Sie die Laufzeit. Was fällt Ihnen auf? 

*Hinweis: Laden Sie zunächst die benötigten Module auf dem bwUniCluster. Setzen Sie dann eine virtuelle Umgebung mit Python auf, in der Sie die benötigten Pythonpakete installieren. Erstellen Sie basierend auf untenstehendem Code ein Python-Skript, welches Sie mithilfe eines Bash-Skripts über SLURM auf dem Cluster submitten (siehe Übung vom 8.11.21). Nachfolgend finden Sie ein Template für das Submit-Skript für den CPU-Job inklusive der benötigten Module. Für die GPU-Nutzung müssen die #SBATCH-Optionen entsprechend angepasst werden. Weitere Informationen dazu finden Sie [hier](https://wiki.bwhpc.de/e/BwUniCluster_2.0_Slurm_common_Features).*

In [ ]:
#!/bin/bash

#SBATCH --job-name=kmeans                  # job name
#SBATCH --partition=single                 # queue for resource allocation
#SBATCH --time=30:00                       # wall-clock time limit  
#SBATCH --mem=30000                        # memory per node
#SBATCH --nodes=1                          # number of nodes to be used
#SBATCH --mail-type=ALL                    # Notify user by email when certain event types occur.
#SBATCH --mail-user=u????@student.kit.edu  # notification email address

module purge                                    # Unload all currently loaded modules.
module load devel/cuda/10.2                     # Load required modules.  
module load compiler/gnu/10.2
module load mpi/openmpi/4.0  
module load lib/hdf5/1.12.0-gnu-10.2-openmpi-4.0  
source <path to your venv folder>/bin/activate  # Activate your virtual environment.

python <path to your python script>

In [ ]:
#!/usr/bin/env python

import h5py
import time
import torch

class KMeans:
    def __init__(self, n_clusters=8, init="random", max_iter=300, tol=-1.0):
        self.init = init             # initialization mode (default: random)
        self.max_iter = max_iter     # maximum number of iterations
        self.n_clusters = n_clusters # number of clusters
        self.tol = tol               # tolerance for convergence criterion

        self._inertia = float("nan")
        self._cluster_centers = None

    def _initialize_centroids(self, x):
        indices = torch.randperm(x.shape[0])[: self.n_clusters]
        self._cluster_centers = x[indices]

    def _fit_to_cluster(self, x):
        distances = torch.cdist(x, self._cluster_centers)
        matching_centroids = distances.argmin(axis=1, keepdim=True)

        return matching_centroids

    def fit(self, x):
        self._initialize_centroids(x)
        new_cluster_centers = self._cluster_centers.clone()

        # Iteratively fit points to centroids.
        for idx in range(self.max_iter):
            # determine the centroids
            print("Iteration", idx, "...")
            matching_centroids = self._fit_to_cluster(x)

            # Update centroids.
            for i in range(self.n_clusters):
                # points in current cluster
                selection = (matching_centroids == i).type(torch.int64)

                # Accumulate points and total number of points in cluster.
                assigned_points = (x * selection).sum(axis=0, keepdim=True)
                points_in_cluster = selection.sum(axis=0, keepdim=True).clamp(
                    1.0, torch.iinfo(torch.int64).max
                )

                # Compute new centroids.
                new_cluster_centers[i : i + 1, :] = assigned_points / points_in_cluster

            # Check whether centroid movement has converged.
            self._inertia = ((self._cluster_centers - new_cluster_centers) ** 2).sum()
            self._cluster_centers = new_cluster_centers.clone()
            if self.tol is not None and self._inertia <= self.tol:
                break

        return self

print("PyTorch k-means clustering")

path = "/pfs/work7/workspace/scratch/ku4408-VL_ScalableAI/data/cityscapes_300.h5"
dataset = "cityscapes_data"

print("Loading data... {}[{}]".format(path, dataset), end="")
print("\n")
# Data is available in HDF5 format.
# An HDF5 file is a container for two kinds of objects:
# - datasets: array-like collections of data
# - groups: folder-like containers holding datasets and other groups
# Most fundamental thing to remember when using h5py is:
# Groups work like dictionaries, and datasets work like NumPy arrays.

# Open file for reading. We use the Cityscapes dataset.

with h5py.File(path, "r") as handle:
    print("Open h5 file...")
    data = torch.tensor(handle[dataset][:300], device="cpu") # default: device ="cpu"; set device="cuda" for GPU
print("Torch tensor created...")

# k-means parameters
num_clusters = 8
num_iterations = 20

kmeans = KMeans(n_clusters=num_clusters, max_iter=num_iterations)
print("Start fitting the data...")
start = time.perf_counter() # Start runtime measurement.
kmeans.fit(data)            # Perform actual k-means clustering.
end = time.perf_counter()   # Stop runtime measurement.
print("DONE.")
print("Run time:","\t{}s".format(end - start), "s")

### Aufgabe 2
Implementieren Sie ausgehend von obigem Code eine Sample-parallele Version des k-Means-Algorithmus. Orientieren Sie sich dabei an der obenstehenden seriellen Implementierung. 
Das Interface bzw. die Benutzung der Klasse im eigentlichen Ausführungsteil des Codes soll gleich bleiben. Für die Parallelisierung benötigen Sie einen entsprechend parallelisierten Dataloader. Diesen finden Sie im untenstehenden Code-Fragment. Testen Sie Ihren Code auf vier Knoten des bwUniClusters. Untenstehend finden Sie ein entsprechendes Submit-Skript in bash.

In [ ]:
#!/bin/bash

#SBATCH --job-name=kmeans                  # job name
#SBATCH --partition=multiple               # queue for the resource allocation.
#SBATCH --time=30:00                       # wall-clock time limit  
#SBATCH --mem=30000                        # memory per node
#SBATCH --nodes=4                          # number of nodes to be used
#SBATCH --cpus-per-task=40                 # number of CPUs required per MPI task
#SBATCH --ntasks-per-node=1                # maximum count of tasks per node
#SBATCH --mail-type=ALL                    # Notify user by email when certain event types occur.
#SBATCH --mail-user=u????@student.kit.edu  # notification email address

export OMP_NUM_THREADS=40

module purge                                    # Unload all currently loaded modules.
module load devel/cuda/10.2                     # Load required modules.  
module load compiler/gnu/10.2
module load mpi/openmpi/4.0  
module load lib/hdf5/1.12.0-gnu-10.2-openmpi-4.0
source <path to your venv folder>/bin/activate  # Activate your virtual environment.

mpirun python <path to your python script>

In [ ]:
#!/usr/bin/env python

import h5py
import time
import torch
from mpi4py import MPI

class KMeans:
    def __init__(self):
        pass
    
# Implementierung Sample-parallele Version HIER.

rank = MPI.COMM_WORLD.rank
size = MPI.COMM_WORLD.size

if rank==0: print("PyTorch k-means clustering")

path = "/pfs/work7/workspace/scratch/ku4408-VL_ScalableAI/data/cityscapes_300.h5"
dataset = "cityscapes_data"

if rank==0: 
    print("Loading data... {}[{}]".format(path, dataset), end="")
    print("\n")
    
with h5py.File(path, "r") as handle:
    chunk = int(handle[dataset].shape[0]/size)
    if rank==size-1: data = torch.tensor(handle[dataset][rank*chunk:])
    else: data = torch.tensor(handle[dataset][rank*chunk:(rank+1)*chunk])

print("\t[OK]")

# k-means parameters
num_clusters = 8
num_iterations = 20

kmeans = KMeans(n_clusters=num_clusters, max_iter=num_iterations)
if rank==0: 
    print("Start fitting the data...")
    start = time.perf_counter() # Start runtime measurement.
    
kmeans.fit(data)            # Perform actual k-means clustering.

if rank==0: 
    end = time.perf_counter()   # Stop runtime measurement.
    print("DONE.")
    print("Run time:","\t{}s".format(end - start), "s")

### Aufgabe 3
Implementieren Sie ausgehend von obigem Code eine Feature-parallele Version des k-Means-Algorithmus. Den entsprechend parallelisierten Dataloader finden Sie im untenstehenden Code-Fragment. Testen Sie Ihren Code auf vier Knoten des bwUniClusters.

In [ ]:
#!/usr/bin/env python

import h5py
import time
import torch
from mpi4py import MPI

class KMeans:
    def __init__(self):
        pass
    
# Implementierung Feature-parallele Version HIER.

rank = MPI.COMM_WORLD.rank
size = MPI.COMM_WORLD.size

if rank==0: print("PyTorch k-means clustering")

path = "/pfs/work7/workspace/scratch/ku4408-VL_ScalableAI/data/cityscapes_300.h5"
dataset = "cityscapes_data"

if rank==0: print("Loading data... {}[{}]".format(path, dataset), end="")

with h5py.File(path, "r") as handle:
    chunk = int(handle[dataset].shape[1]/size)
    if rank==size-1: data = torch.tensor(handle[dataset][:,rank*chunk:])
    else: data = torch.tensor(handle[dataset][:,rank*chunk:(rank+1)*chunk])

print("\t[OK]")

# k-means parameters
num_clusters = 8
num_iterations = 20

kmeans = KMeans(n_clusters=num_clusters, max_iter=num_iterations)

if rank==0: 
    print("Start fitting the data...")
    start = time.perf_counter() # Start runtime measurement.

kmeans.fit(data)            # Perform actual k-means clustering.

if rank==0:
    end = time.perf_counter()   # Stop runtime measurement.
    print("DONE.")
    print("Run time:","\t{}s".format(end - start), "s")